In [1]:
# importing some libraries
import numpy as np
import pandas as pd
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("MovieRec").getOrCreate()
sc = spark.sparkContext

#Read in data
ratings_rdd = sc.textFile('D:/pyproj/data/ml-1m/ratings.dat')\
            .map(lambda x: x.split("::"))
movies_rdd = sc.textFile('D:/pyproj/data/ml-1m/movies.dat').map(lambda x: x.split("::"))
users_rdd = sc.textFile('D:/pyproj/data/ml-1m/users.dat').map(lambda x:x.split("::"))


In [3]:
# ratings_rdd.collect()
# movies_rdd.collect()
# users_rdd.collect()
#Create DataFrames
ratings_df = ratings_rdd.toDF(["UserID","MovieID","Rating","Timestamp"])
ratings_df.show()

+------+-------+------+---------+
|UserID|MovieID|Rating|Timestamp|
+------+-------+------+---------+
|     1|   1193|     5|978300760|
|     1|    661|     3|978302109|
|     1|    914|     3|978301968|
|     1|   3408|     4|978300275|
|     1|   2355|     5|978824291|
|     1|   1197|     3|978302268|
|     1|   1287|     5|978302039|
|     1|   2804|     5|978300719|
|     1|    594|     4|978302268|
|     1|    919|     4|978301368|
|     1|    595|     5|978824268|
|     1|    938|     4|978301752|
|     1|   2398|     4|978302281|
|     1|   2918|     4|978302124|
|     1|   1035|     5|978301753|
|     1|   2791|     4|978302188|
|     1|   2687|     3|978824268|
|     1|   2018|     4|978301777|
|     1|   3105|     5|978301713|
|     1|   2797|     4|978302039|
+------+-------+------+---------+
only showing top 20 rows



In [4]:
movies_df = movies_rdd.toDF(["MovieID","Title","Genres"])
movies_df.show()

+-------+--------------------+--------------------+
|MovieID|               Title|              Genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Animation|Childre...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|        Comedy|Drama|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|Adventure|Children's|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Animation|Children's|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|      Drama|Thriller|
|     17|Sen

In [5]:
users_df = users_rdd.toDF(["UserID","Gender","Age","Occupation","Zip-code"])
users_df.show()



+------+------+---+----------+--------+
|UserID|Gender|Age|Occupation|Zip-code|
+------+------+---+----------+--------+
|     1|     F|  1|        10|   48067|
|     2|     M| 56|        16|   70072|
|     3|     M| 25|        15|   55117|
|     4|     M| 45|         7|   02460|
|     5|     M| 25|        20|   55455|
|     6|     F| 50|         9|   55117|
|     7|     M| 35|         1|   06810|
|     8|     M| 25|        12|   11413|
|     9|     M| 25|        17|   61614|
|    10|     F| 35|         1|   95370|
|    11|     F| 25|         1|   04093|
|    12|     M| 25|        12|   32793|
|    13|     M| 45|         1|   93304|
|    14|     M| 35|         0|   60126|
|    15|     M| 25|         7|   22903|
|    16|     F| 35|         0|   20670|
|    17|     M| 50|         1|   95350|
|    18|     F| 18|         3|   95825|
|    19|     M|  1|        10|   48073|
|    20|     M| 25|        14|   55113|
+------+------+---+----------+--------+
only showing top 20 rows



In [ ]:
#################################################################################
###Traditional Mechine Learning Method with ALS Recommendation System############
#################################################################################


In [6]:
#Build DataFrame
Rec1_df = movies_df.join(ratings_df,movies_df.MovieID == ratings_df.MovieID)
#Rec1_df.show()
from pyspark.sql.types import *
Rec1_df = Rec1_df.select("UserID","Title","Rating")
Rec1_df = Rec1_df.withColumn("UserID",Rec1_df["UserID"].cast(IntegerType()))
Rec1_df = Rec1_df.withColumn("Rating",Rec1_df["Rating"].cast(IntegerType()))
Rec1_df.show()

+------+--------------+------+
|UserID|         Title|Rating|
+------+--------------+------+
|     2|Platoon (1986)|     2|
|    19|Platoon (1986)|     3|
|    23|Platoon (1986)|     3|
|    24|Platoon (1986)|     4|
|    26|Platoon (1986)|     3|
|    33|Platoon (1986)|     3|
|    36|Platoon (1986)|     5|
|    48|Platoon (1986)|     3|
|    53|Platoon (1986)|     5|
|    57|Platoon (1986)|     4|
|    58|Platoon (1986)|     5|
|    62|Platoon (1986)|     5|
|    65|Platoon (1986)|     4|
|    66|Platoon (1986)|     4|
|    88|Platoon (1986)|     3|
|   107|Platoon (1986)|     4|
|   116|Platoon (1986)|     5|
|   117|Platoon (1986)|     3|
|   118|Platoon (1986)|     3|
|   123|Platoon (1986)|     4|
+------+--------------+------+
only showing top 20 rows



In [7]:
from pyspark.sql.functions import *
print((Rec1_df.count(),len(Rec1_df.columns)))
Rec1_df.printSchema()


(1000209, 3)
root
 |-- UserID: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Rating: integer (nullable = true)



In [9]:
#Check the num of rating of each user
Rec1_df.groupBy("UserID").count().orderBy('count',ascending= False).show()

+------+-----+
|UserID|count|
+------+-----+
|  4169| 2314|
|  1680| 1850|
|  4277| 1743|
|  1941| 1595|
|  1181| 1521|
|   889| 1518|
|  3618| 1344|
|  2063| 1323|
|  1150| 1302|
|  1015| 1286|
|  5795| 1277|
|  4344| 1271|
|  1980| 1260|
|  2909| 1258|
|  1449| 1243|
|  4510| 1240|
|   424| 1226|
|  4227| 1222|
|  5831| 1220|
|  3841| 1216|
+------+-----+
only showing top 20 rows



In [10]:
#Check the number of times movie been rated
Rec1_df.groupBy("Title").count().orderBy('count',ascending=False).show()

+--------------------+-----+
|               Title|count|
+--------------------+-----+
|American Beauty (...| 3428|
|Star Wars: Episod...| 2991|
|Star Wars: Episod...| 2990|
|Star Wars: Episod...| 2883|
|Jurassic Park (1993)| 2672|
|Saving Private Ry...| 2653|
|Terminator 2: Jud...| 2649|
|  Matrix, The (1999)| 2590|
|Back to the Futur...| 2583|
|Silence of the La...| 2578|
| Men in Black (1997)| 2538|
|Raiders of the Lo...| 2514|
|        Fargo (1996)| 2513|
|Sixth Sense, The ...| 2459|
|   Braveheart (1995)| 2443|
|Shakespeare in Lo...| 2369|
|Princess Bride, T...| 2318|
|Schindler's List ...| 2304|
|L.A. Confidential...| 2288|
|Groundhog Day (1993)| 2278|
+--------------------+-----+
only showing top 20 rows



In [11]:
#Data Processing
#import String indexer to convert string value to numeric values
from pyspark.ml.feature import StringIndexer,IndexToString
stringIndexer = StringIndexer(inputCol="Title",outputCol="Title_index")
model = stringIndexer.fit(Rec1_df)
indexed = model.transform(Rec1_df)
indexed.show()

+------+--------------+------+-----------+
|UserID|         Title|Rating|Title_index|
+------+--------------+------+-----------+
|     2|Platoon (1986)|     2|      145.0|
|    19|Platoon (1986)|     3|      145.0|
|    23|Platoon (1986)|     3|      145.0|
|    24|Platoon (1986)|     4|      145.0|
|    26|Platoon (1986)|     3|      145.0|
|    33|Platoon (1986)|     3|      145.0|
|    36|Platoon (1986)|     5|      145.0|
|    48|Platoon (1986)|     3|      145.0|
|    53|Platoon (1986)|     5|      145.0|
|    57|Platoon (1986)|     4|      145.0|
|    58|Platoon (1986)|     5|      145.0|
|    62|Platoon (1986)|     5|      145.0|
|    65|Platoon (1986)|     4|      145.0|
|    66|Platoon (1986)|     4|      145.0|
|    88|Platoon (1986)|     3|      145.0|
|   107|Platoon (1986)|     4|      145.0|
|   116|Platoon (1986)|     5|      145.0|
|   117|Platoon (1986)|     3|      145.0|
|   118|Platoon (1986)|     3|      145.0|
|   123|Platoon (1986)|     4|      145.0|
+------+---

In [12]:
#Split the data into train and test dataset
train,test = indexed.randomSplit([0.75,0.25])
print("train:",train.count()," ","test:",test.count())



train: 749657   test: 250552


In [13]:
#ALS Recommender System
from pyspark.ml.recommendation import ALS
rec = ALS(maxIter=10,regParam=0.01,userCol="UserID",itemCol="Title_index",ratingCol="Rating",nonnegative=True,coldStartStrategy="drop")
#Fit the model on train
rec_model = rec.fit(train)
#Making predictions on test set
predicted_ratings = rec_model.transform(test)
predicted_ratings.printSchema()

root
 |-- UserID: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Rating: integer (nullable = true)
 |-- Title_index: double (nullable = false)
 |-- prediction: float (nullable = false)



In [14]:
predicted_ratings.orderBy(rand()).show()



+------+--------------------+------+-----------+----------+
|UserID|               Title|Rating|Title_index|prediction|
+------+--------------------+------+-----------+----------+
|  5345|Slums of Beverly ...|     5|     1160.0| 3.7778513|
|  5749|  Killing Zoe (1994)|     1|     1545.0| 3.1019611|
|  3598|Shaggy Dog, The (...|     1|     1659.0|  0.975405|
|  1569|  Sling Blade (1996)|     4|      135.0|  4.793153|
|   927|Big Lebowski, The...|     3|      167.0| 2.9251337|
|  4543|     Grease 2 (1982)|     2|     1042.0| 1.5661153|
|  6007|Blues Brothers, T...|     2|       98.0|  3.199599|
|  3985|      Georgia (1995)|     2|     2576.0| 3.1793745|
|  2156|Crocodile Dundee ...|     4|      772.0|  2.089546|
|   698| Shining, The (1980)|     5|      171.0|  5.083621|
|  2755|        Fargo (1996)|     5|       12.0| 2.7639518|
|  3261|        Blade (1998)|     4|      306.0| 3.3229928|
|  4064|My Life in Pink (...|     3|     1418.0| 3.6887548|
|  5685|       Clerks (1994)|     4|    

In [15]:
#Measure RMSE
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(metricName="rmse",predictionCol='prediction',labelCol="Rating")
rmse = evaluator.evaluate(predicted_ratings)
print(rmse)

0.8836925253520499


In [ ]:
#####################################################################
################     New Method   ##############################
#####################################################################


In [ ]:
#Data Processing
movies_df.show()
movies_df_2 = movies_df.withColumn("Genres",split(movies_df["Genres"],"\|"))
movies_df_2.show()
Rec2_df = movies_df_2.join(ratings_df,['MovieID']).join(users_df,['UserID'])

In [ ]:
Rec2_df.show()
# Rec_df = Rec2_df.drop("UserID","MovieID")
# Rec_df.show()


In [ ]:
#####################################################################
#######################   Hang_Rec ##################################
#####################################################################





In [ ]:
value_set = Rec2_df.select("MovieID").distinct().rdd.map(lambda r:r[0]).collect()
ratings_data = Rec2_df.groupBy("UserID").pivot("MovieID",value_set).agg(max("Rating")).fillna(0)
ratings_data.show()

In [ ]:
ratings_data1 = ratings_data.na.fill(0).orderBy("UserID")
ratings_data1.show()
